In [6]:
#let's import somre relavent libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup


import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [7]:
data_cat_list = [
    "demographics",
    "dietary", 
    "examination", 
    "laboratory", 
    "questionnaire", 
    "limitedaccess"
]

In [8]:
from NHANES_data_API import NHANESDataAPI

api = NHANESDataAPI('demographics')

['Demographic Variables & Sample Weights'
 'Demographic Variables and Sample Weights']


In [10]:
api.variable_table

,Variable Name,Variable Description,Data File Name,Data File Description,Years
0,AIALANG,Language of the MEC ACASI Interview Instrument,DEMO_D,Demographic Variables & Sample Weights,2005-2006
1,DMDBORN,In what country {were you/was SP} born?,DEMO_D,Demographic Variables & Sample Weights,2005-2006
2,DMDCITZN,{Are you/Is SP} a citizen of the United States...,DEMO_D,Demographic Variables & Sample Weights,2005-2006
3,DMDEDUC2,(SP Interview Version) What is the highest gra...,DEMO_D,Demographic Variables & Sample Weights,2005-2006
4,DMDEDUC3,(SP Interview Version) What is the highest gra...,DEMO_D,Demographic Variables & Sample Weights,2005-2006
...,...,...,...,...,...
537,SIAINTRP,Was an interpreter used to conduct the Sample ...,DEMO_J,Demographic Variables and Sample Weights,2017-2018
538,SIALANG,Language of the Sample Person Interview Instru...,DEMO_J,Demographic Variables and Sample Weights,2017-2018
539,SIAPROXY,Was a Proxy respondent used in conducting the ...,DEMO_J,Demographic Variables and Sample Weights,2017-2018
540,WTINT2YR,Full sample 2 year interview weight.,DEMO_J,Demographic Variables and Sample Weights,2017-2018


In [14]:
common_variables, variable_cycles_dict = api.common_variables(cycle_years= ['1999-2000'])

In [16]:
common_variables

['WTIREP09',
 'WTIREP05',
 'INDFMPIR',
 'SDJ1REPN',
 'WTIREP41',
 'WTIREP10',
 'WTMREP51',
 'WTIREP30',
 'INDHHINC',
 'WTMREP02',
 'WTMREP43',
 'WTMREP08',
 'WTIREP47',
 'RIDRETH2',
 'WTIREP08',
 'WTIREP14',
 'WTMREP04',
 'WTIREP23',
 'WTMREP05',
 'WTMREP23',
 'WTMREP44',
 'WTIREP07',
 'WTMREP33',
 'WTMREP46',
 'WTMREP30',
 'RIDPREG',
 'WTIREP20',
 'WTIREP19',
 'WTMREP16',
 'WTMREP22',
 'DMDCITZN',
 'WTMREP26',
 'WTIREP17',
 'DMDSCHOL',
 'WTIREP04',
 'WTMREP14',
 'WTMREP37',
 'WTMEC2YR',
 'WTIREP49',
 'RIDEXPRG',
 'WTIREP22',
 'RIDEXMON',
 'WTMREP21',
 'WTIREP26',
 'WTMREP45',
 'WTIREP11',
 'WTMREP49',
 'INDFMINC',
 'WTMREP11',
 'WTIREP28',
 'WTIREP37',
 'WTIREP06',
 'RIDAGEMN',
 'WTMREP01',
 'WTMREP13',
 'RIDRETH1',
 'WTIREP33',
 'RIDAGEEX',
 'WTIREP21',
 'WTMREP03',
 'WTIREP36',
 'DMDHREDU',
 'DMDHRBRN',
 'DMDMARTL',
 'WTMREP17',
 'WTIREP42',
 'WTMREP40',
 'WTIREP12',
 'WTIREP43',
 'WTMREP19',
 'WTMREP18',
 'WTIREP44',
 'SDDSRVYR',
 'DMDHHSIZ',
 'DMDHRGND',
 'WTMREP47',
 'WTIREP38',


In [21]:
cycle_list = [  
                '1999-2000',
                '2001-2002',
                '2003-2004',
                '2005-2006',
                '2007-2008',
                '2009-2010',
                '2011-2012',
                '2013-2014',
                '2015-2016',
                '2017-2018']
data_cat_list = [
                "demographics",
                "dietary", 
                "examination", 
                "laboratory", 
                "questionnaire", 
                "limitedaccess"
            ]

In [22]:
def check_cycle(input_cycle):
    """
    Check the validity of a cycle and return valid cycle(s) based on input.

    Args:
    input_cycle (str): The input cycle year or range.

    Returns:
    list: List of valid cycle(s) based on input.
    """
    if '-' in input_cycle:
        start_year, end_year = input_cycle.split('-')
        the_cyclelist = check_in_between_cycle(start_year, end_year, cycle_list)
        return the_cyclelist
    elif input_cycle in cycle_list:
        return [input_cycle]
    elif any(input_cycle in cycles for cycles in cycle_list):
        return [cycle for cycle in cycle_list if input_cycle in cycle]
    else:
        return []

def check_in_between_cycle(start_year, end_year, cycle_list):
    """
    Check for valid cycles within a range.

    Args:
    start_year (str): The start year of the range.
    end_year (str): The end year of the range.
    cycle_list (list): List of available cycle years.

    Returns:
    list: List of valid cycle(s) within the range.
    """
    list_of_cycles_to_be_worked_on = []
    flager = 0
    for cycle in cycle_list:
        if start_year in cycle:
            flager = 1
        if flager == 1:
            list_of_cycles_to_be_worked_on.append(cycle)
        if end_year in cycle:
            return list_of_cycles_to_be_worked_on
    return list_of_cycles_to_be_worked_on


In [26]:
def common_variables(cycle_years):
    """
    Find common variables across multiple cycle years and create a dictionary with variable-cycles mapping.

    Args:
    cycle_years (str or list of str): Either a single cycle year or a list of cycle years.

    Returns:
    list: List of common variables.
    list: List of uncommon variables.
    dict: A dictionary of {variable: [cycles]}.
    """
    if isinstance(cycle_years, str):
        cycle_years = [cycle_years]


    common_variables = None
    variable_cycles_dict = {}
    valid_cycles = list()
    for cycle in cycle_years:
        valid_cycles = valid_cycles + check_cycle(cycle)

        
    return valid_cycles

In [25]:
check_cycle("1999-2004")

['1999-2000', '2001-2002', '2003-2004']

In [27]:
common_variables("2001-2002")

['2001-2002']

In [28]:
common_variables(["1999-2000", "2001-2002"])

['1999-2000', '2001-2002']

In [29]:
common_variables(["1999-2004"])

['1999-2000', '2001-2002', '2003-2004']

In [30]:
common_variables(["1999-2000", "2001-2002"])

['1999-2000', '2001-2002']

In [31]:
common_variables('2007')

[]